<a href="https://colab.research.google.com/github/leoffx/deep-learning-model-for-computational-fluid-dynamics/blob/master/dl_model_for_cfd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lattice Boltzmann Method

In [ ]:
import numpy as np
from tqdm.notebook import tqdm

np.random.seed(42)

In [ ]:
class LBM:
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.u = np.zeros((height, width, 2))
        self.fin = np.zeros((height, width, 9))
        self.omega = 1.9 #proportional to viscosity
        self.generateObj()

        u0 = 0.08 #inlet boundary condition for velocity
        self.fin[..., 0] = 4. / 9. * (1 - 1.5 * u0**2)
        self.fin[..., 2] = 1. / 9. * (1 - 1.5 * u0**2)
        self.fin[..., 4] = 1. / 9. * (1 - 1.5 * u0**2)
        self.fin[..., 1] = 1. / 9. * (1 + 3 * u0 + 4.5 * u0**2 - 1.5 * u0**2)
        self.fin[..., 3] = 1. / 9. * (1 - 3 * u0 + 4.5 * u0**2 - 1.5 * u0**2)
        self.fin[..., 5] = 1. / 36. * (1 + 3 * u0 + 4.5 * u0**2 - 1.5 * u0**2)
        self.fin[..., 8] = 1. / 36. * (1 + 3 * u0 + 4.5 * u0**2 - 1.5 * u0**2)
        self.fin[..., 6] = 1. / 36. * (1 - 3 * u0 + 4.5 * u0**2 - 1.5 * u0**2)
        self.fin[..., 7] = 1. / 36. * (1 - 3 * u0 + 4.5 * u0**2 - 1.5 * u0**2)
        self.rho = np.sum(self.fin, axis=2)

        self.fin1 = self.fin[0, 0, 1]
        self.fin5 = self.fin[0, 0, 5]
        self.fin8 = self.fin[0, 0, 8]
        self.fin6 = self.fin[0, 0, 6]
        self.fin7 = self.fin[0, 0, 7]
        self.fin3 = self.fin[0, 0, 3]

    def collision(self):
        self.rho = np.sum(self.fin, axis=2)

        self.u[..., 0] = (self.fin[..., 1] + self.fin[..., 5] +
                           self.fin[..., 8] - self.fin[..., 3] -
                           self.fin[..., 7] - self.fin[..., 6]) / self.rho
        self.u[..., 1] = (self.fin[..., 2] + self.fin[..., 5] +
                           self.fin[..., 6] - self.fin[..., 4] -
                           self.fin[..., 7] - self.fin[..., 8]) / self.rho

        u2 = self.u[..., 0]**2 + self.u[..., 1]**2
        uxuy = self.u[..., 0] * self.u[..., 1]
        um32u2 = 1. - 1.5 * u2  #1 minus 3/2 of u**2

        self.fin[..., 0] = self.fin[..., 0] * (
            1. - self.omega) + self.omega * 4. / 9. * self.rho * (um32u2)
        self.fin[..., 1] = self.fin[..., 1] * (
            1. - self.omega) + self.omega * 1. / 9. * self.rho * (
                3. * self.u[..., 0] + 4.5 * self.u[..., 0]**2 + um32u2)
        self.fin[..., 2] = self.fin[..., 2] * (
            1. - self.omega) + self.omega * 1. / 9. * self.rho * (
                3. * self.u[..., 1] + 4.5 * self.u[..., 1]**2 + um32u2)
        self.fin[..., 3] = self.fin[..., 3] * (
            1. - self.omega) + self.omega * 1. / 9. * self.rho * (
                -3. * self.u[..., 0] + 4.5 * self.u[..., 0]**2 + um32u2)
        self.fin[..., 4] = self.fin[..., 4] * (
            1. - self.omega) + self.omega * 1. / 9. * self.rho * (
                -3. * self.u[..., 1] + 4.5 * self.u[..., 1]**2 + um32u2)
        self.fin[..., 5] = self.fin[..., 5] * (
            1. - self.omega) + self.omega * 1. / 36. * self.rho * (
                3. * (self.u[..., 0] + self.u[..., 1]) + 4.5 *
                (u2 + 2. * uxuy) + um32u2)
        self.fin[..., 6] = self.fin[..., 6] * (
            1. - self.omega) + self.omega * 1. / 36. * self.rho * (
                3. * (-self.u[..., 0] + self.u[..., 1]) + 4.5 *
                (u2 - 2. * uxuy) + um32u2)
        self.fin[..., 7] = self.fin[..., 7] * (
            1. - self.omega) + self.omega * 1. / 36. * self.rho * (
                3. * (-self.u[..., 0] - self.u[..., 1]) + 4.5 *
                (u2 + 2. * uxuy) + um32u2)
        self.fin[..., 8] = self.fin[..., 8] * (
            1. - self.omega) + self.omega * 1. / 36. * self.rho * (
                3. * (self.u[..., 0] - self.u[..., 1]) + 4.5 *
                (u2 - 2. * uxuy) + um32u2)

        self.fin[..., 1][:, 0] = self.fin1
        self.fin[..., 3][:, 0] = self.fin3
        self.fin[..., 5][:, 0] = self.fin5
        self.fin[..., 8][:, 0] = self.fin8
        self.fin[..., 6][:, 0] = self.fin6
        self.fin[..., 7][:, 0] = self.fin7

    def streaming(self):
        self.fin[..., 2] = np.roll(self.fin[..., 2], 1, axis=0)
        self.fin[..., 5] = np.roll(self.fin[..., 5], 1, axis=0)
        self.fin[..., 6] = np.roll(self.fin[..., 6], 1, axis=0)

        self.fin[..., 4] = np.roll(self.fin[..., 4], -1, axis=0)
        self.fin[..., 8] = np.roll(self.fin[..., 8], -1, axis=0)
        self.fin[..., 7] = np.roll(self.fin[..., 7], -1, axis=0)

        self.fin[..., 1] = np.roll(self.fin[..., 1], 1, axis=1)
        self.fin[..., 5] = np.roll(self.fin[..., 5], 1, axis=1)
        self.fin[..., 8] = np.roll(self.fin[..., 8], 1, axis=1)

        self.fin[..., 3] = np.roll(self.fin[..., 3], -1, axis=1)
        self.fin[..., 7] = np.roll(self.fin[..., 7], -1, axis=1)
        self.fin[..., 6] = np.roll(self.fin[..., 6], -1, axis=1)

        #bounceback boundary
        self.fin[..., 2][self.objBoundary[..., 2]] = self.fin[..., 4][self.obj]
        self.fin[..., 4][self.objBoundary[..., 4]] = self.fin[..., 2][self.obj]
        self.fin[..., 1][self.objBoundary[..., 1]] = self.fin[..., 3][self.obj]
        self.fin[..., 3][self.objBoundary[..., 3]] = self.fin[..., 1][self.obj]
        self.fin[..., 5][self.objBoundary[..., 5]] = self.fin[..., 7][self.obj]
        self.fin[..., 6][self.objBoundary[..., 6]] = self.fin[..., 8][self.obj]
        self.fin[..., 8][self.objBoundary[..., 8]] = self.fin[..., 6][self.obj]
        self.fin[..., 7][self.objBoundary[..., 7]] = self.fin[..., 5][self.obj]

    def generateObj(self):
        self.obj = np.zeros((self.height, self.width), bool)
        self.objBoundary = np.zeros((self.height, self.width, 9), bool)

        # generate a random number of objects between 3 and 12
        numCircle = np.random.randint(3, 12)
        for _ in range(numCircle):
            # chooses circle radius
            radCircle = np.random.randint(5, 10)
            # chooses a random area where the object will be placed,
            # limited to not be too close to the margin
            xCenter = np.random.randint(5 + radCircle, .9 * self.width - radCircle)
            yCenter = np.random.randint(5 + radCircle, self.height - radCircle - 5)
            # loop through all pixels, if it's inside the object area, set the matrix position to True
            for i in range(self.height):
                for j in range(self.width):
                    # general circle equation, can also be changed for other shapes
                    if (i - yCenter)**2 + (j - xCenter)**2 <= radCircle**2:
                        self.obj[i, j] = True

        self.obj[0,:] = True
        self.objBoundary[..., 2] = np.roll(self.obj, 1, axis=0)
        self.objBoundary[..., 4] = np.roll(self.obj, -1, axis=0)
        self.objBoundary[..., 1] = np.roll(self.obj, 1, axis=1)
        self.objBoundary[..., 3] = np.roll(self.obj, -1, axis=1)
        self.objBoundary[..., 5] = np.roll(self.objBoundary[..., 2], 1, axis=1)
        self.objBoundary[..., 6] = np.roll(self.objBoundary[..., 2], -1, axis=1)
        self.objBoundary[..., 8] = np.roll(self.objBoundary[..., 4], 1, axis=1)
        self.objBoundary[..., 7] = np.roll(self.objBoundary[..., 4], -1, axis=1)

In [ ]:
def create_examples(name, examples_number, simulation_res=128):
    print(f'Creating {name} dataset.')
    trainData = np.zeros((examples_number*100, simulation_res, simulation_res, 9))
    objData = np.zeros((examples_number*100, simulation_res, simulation_res, 1))
    x = 0
    for i in tqdm(range(examples_number)):
        fluid = LBM(simulation_res, simulation_res)
        for _ in range(40*120):  #warm up
            fluid.streaming()
            fluid.collision()
        for j in range(100):
            for _ in range(60):
                fluid.streaming()
                fluid.collision()
                
            trainData[x] = fluid.fin  
            objData[x, ..., 0] = fluid.obj
            x+=1

    np.save(f'x{name}', trainData)
    np.save(f'obj{name}', objData)

In [ ]:
create_examples('train', examples_number=30)
create_examples('test', examples_number=3)

If you don't want to have to run this part everytime, you can also connect the notebook to Google Drive and save the files there using Colab's integration.

# Deep Learning Model

In [ ]:
!git clone https://github.com/leoffx/deep-learning-model-for-computational-fluid-dynamics.git DLcfd

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model

import DLcfd.utils as utils  # file with helper functions
import matplotlib.pyplot as plt

np.random.seed(42)
tf.random.set_seed(42)

**Loading the Dataset**

In [ ]:
x_train = np.load('xtrain.npy')
x_train = (x_train - x_train.min())/(x_train.max()-x_train.min())
obj_train = np.load('objtrain.npy')

In [ ]:
x_test = np.load('xtest.npy')
x_test = (x_test - x_test.min())/(x_test.max()-x_test.min())
obj_test = np.load('objtest.npy')

Default residual block.

<center><img src="figures/residual_block.png" width="400" /></center>

In [ ]:
def residual_block(X, num_filters, stride_1, stride_2=1, kernel_size_1=3, kernel_size_2=3, padding='VALID'):

    X_shortcut = X

    X = L.Conv2D(filters=num_filters, kernel_size=kernel_size_1, strides=stride_1,
                padding=padding, activation='swish')(X)
    X = L.Conv2D(filters=num_filters, kernel_size=kernel_size_2, strides=stride_2,
               padding='SAME', activation='swish')(X)

    if padding == 'VALID':
        X_shortcut = L.Conv2D(
            filters=num_filters, kernel_size=kernel_size_1, strides=stride_1,
            activation='swish')(X_shortcut)
    else:
        X_shortcut = L.Conv2D(filters=num_filters,
                            kernel_size=1, strides=1,)(X_shortcut)

    X = L.Add()([X, X_shortcut])
    X = L.Activation('swish')(X)

    return X

The encoder block is divided in two parts, one for the simulation tensor encoding, and the other one for the object tensor encoding. The encoder reduces the simulation size to make the computation of the full simulation more efficient.

<center><img src="figures/encoder.png" width="500" /></center>

In [ ]:
def encoder_state(input_shape=(None, None, 9)):

    X_input = L.Input(input_shape)

    X = X_input
    X = residual_block(X, 16, stride_1=1, kernel_size_1=4, padding='SAME')
    X = residual_block(X, 32, stride_1=2, kernel_size_1=4)
    X = residual_block(X, 32, stride_1=1, padding='SAME')
    X = residual_block(X, 32, stride_1=1, padding='SAME')
    X = residual_block(X, 64, stride_1=2, kernel_size_1=4)
    X = residual_block(X, 64, stride_1=1, padding='SAME')
    X = residual_block(X, 64, stride_1=1, padding='SAME')

    obj_input = L.Input((None, None, 1))

    Y = obj_input
    Y = residual_block(Y, 32, stride_1=2, kernel_size_1=4)
    Y = residual_block(Y, 32, stride_1=1, padding='SAME')
    Y = residual_block(Y, 64, stride_1=2, kernel_size_1=4)
    Y = residual_block(Y, 64, stride_1=1, padding='SAME')
    b_add = residual_block(Y, 64, stride_1=1, padding='SAME')
    b_mul = residual_block(Y, 64, stride_1=1, padding='SAME')
    X = L.Multiply()([X, b_mul])
    X = L.Add()([X, b_add])

    model = Model(inputs=[X_input, obj_input], outputs=[X, b_add, b_mul])

    return model

The decoder block upsamples the compressed simulation back to its original dimension. 

<center><img src="figures/decoder.png" width="200" /></center>

In [ ]:
def decoder_state(input_shape):

    X_input = L.Input(input_shape)

    X = X_input
    X = L.Conv2DTranspose(64, kernel_size=4, strides=2)(X)
    X = residual_block(X, 32, stride_1=1, padding='SAME')
    X = residual_block(X, 32, stride_1=1, padding='SAME')
    X = L.Conv2DTranspose(32, kernel_size=4, strides=2)(X)
    X = residual_block(X, 16, stride_1=1, padding='SAME')
    X = residual_block(X, 16, stride_1=1, padding='SAME')
    X = L.Conv2DTranspose(9, kernel_size=3, strides=1)(X)

    model = Model(inputs=X_input, outputs=X)

    return model

The temporal evolution, or compression mapping block, is responsible for advancing the simulation in time, returning its next frame in the latent compressed dimensions.

<center><img src="figures/mapping.png" width="300" /></center>

In [ ]:
def compression_mapping(input_shape_X, input_shape_b):

    X_input = L.Input(input_shape_X)
    b_add = L.Input(input_shape_b)
    b_mul = L.Input(input_shape_b)

    X = X_input
    X = residual_block(X, 64, stride_1=1, padding='SAME')
    X = residual_block(X, 64, stride_1=1, padding='SAME')
    X = residual_block(X, 64, stride_1=1, padding='SAME')
    X = residual_block(X, 64, stride_1=1, padding='SAME')

    X = L.Multiply()([X, b_mul])
    X = L.Add()([X, b_add])

    model = Model(inputs=[X_input, b_add, b_mul], outputs=X)

    return model

The final model is created using the previously defined parts. To help guaranteeing the long term temporal convergence of the neural network, during training the 5 next simulation frames are predicted, reusing the decoder and temporal evolution parts.  

<center><img src="figures/overview.png" width="800" /></center>

In [ ]:
def create_final_model(model_encoder_state, model_decoder_state, model_compression_mapping):
    X_input = L.Input((None, None, 9))
    obj_input = L.Input((None, None, 1))
    outputs = []

    compressed, b_add, b_mul = model_encoder_state([X_input, obj_input])

    for _ in range(5):
        outputs.append(model_decoder_state([compressed]))
        compressed = model_compression_mapping(
            [compressed, b_add, b_mul])

    model = Model(inputs=[X_input, obj_input], outputs=outputs)

    return model

In [ ]:
model_encoder_state = encoder_state()
model_decoder_state = decoder_state(model_encoder_state.output_shape[0][1:])
model_compression_mapping = compression_mapping(
    model_encoder_state.output_shape[0][1:], model_encoder_state.output_shape[1][1:])
model_parts = [model_encoder_state, model_decoder_state, model_compression_mapping]

In [ ]:
model = create_final_model(
    model_encoder_state, model_decoder_state, model_compression_mapping)

**Training**

Although the loss being minimized is the Mean Squared Error (MSE) loss, it isn't very informative about the simulation convergence on the long term. To check the convergence during training, a callback that computes a full 100-frames simulation, and display the last frame in the end of every epoch is used.

In [ ]:
class visualize_result(tf.keras.callbacks.Callback):
    def on_epoch_end(self, batch, logs={}):
        predic = utils.generate_simulation(
            model_parts, x_test[0:1], obj_test[0:1], frame_num=100)
        vel = utils.get_velocity(predic[-1])
        u_plot = np.abs(vel[0, :, :, 0], vel[0, :, :, 1])
        plt.imshow(u_plot, cmap='viridis')
        plt.show()
class save_models(tf.keras.callbacks.Callback):
    def on_epoch_end(self, batch, logs={}):
        model_list.append(model.get_weights())

In [ ]:
model_list = []
sav = save_models()
vis = visualize_result()

In [ ]:
optm = tf.optimizers.Adam(learning_rate=5e-4)
model.compile(optimizer=optm, loss='mse')

In [ ]:
model.fit(x=[x_train[:-5], obj_train[:-5]], y=[x_train[i:i-5]
                                               for i in range(5)], batch_size=4, epochs=20, callbacks=[vis, sav])

**Visualization**

In [ ]:
#model.set_weights(model_list[i]) #choose the model that produced the best result

In [ ]:
y_hat = utils.generate_simulation(model_parts, x_test[0:1], obj_test[0:1])

In [ ]:
utils.make_animation(y_hat, [], 'vel')